In [1]:
from import_data import *
from noise_models import *
from PICDNN_extended import *
from Branch_flow_xy_split import *
# from generate_xy import *
import numpy as np
import matplotlib.pyplot as plt
# from volt_to_p_est import *
# from VI_to_P import *
# from linear_models import *
from feature_bins import std_bins_bflow
from IPython.display import clear_output

#%% importing single hour imag dataset
VR, VI, bflow, Pinj, Qinj, CFR, CFI, CTR, CTI = import_curr_branch_data()

In [2]:
def data(addbus=[], output = "vi_all"):
#%% Getting X and Y 
    x_vr, x_vi, x_cr, x_ci,  ytot = Branch_flow_custom_xy_split(bflow, VR, VI, CFR, CFI, CTR, CTI, Pinj, Qinj, output = output,  num_bus = 11, add_bus = addbus)
    #%% Adding noise 
    x_vr_n, x_vi_n = GMM_real_imag(x_vr, x_vi)
    x_cr_n, x_ci_n = GMM_real_imag(x_cr, x_ci)
    # bflow_n = noise_gaussian(X, range1, mu1)
    # bflow_n = NoisyMag(x_bflow.values)
 

    # X_p = VI_to_P(x_vm_n, x_va_n, x_cm_n, x_ca_n, num_bus = 11, add_bus = [])
    X_v = np.concatenate((x_vr_n, x_vi_n, x_cr_n, x_ci_n), axis=1)
    Y_p = ytot
    return X_v, Y_p

In [3]:
bdata = np.genfromtxt(r"System_data\bdata.csv", delimiter=',')

In [14]:
def one_run(add_bus):
    X_v, Y_p = data(add_bus, 'vi_all')
    dnn = Deep_Network(layers=5, nodes=int(X_v.shape[1]*1.25), lr = 1e-4, epoch = 200,
                    batch_norm = True, dropout = 0)
    # dnn.y_normalise = True
    dnn.loss = "mse"
    mse, mape, r2 = dnn.model_parse(X_v,Y_p,Pinj, Qinj, ntest=1)
    return X_v, Y_p, dnn, mse, mape, r2



In [7]:
def create_A_array(bdata, n_buses):
    n = bdata.shape[0]
    A = np.zeros((n_buses, n))
    for i in range(n_buses):
        for j in range(n):
            if bdata[j, 0] == i+1:
                A[i, j] = 1
    return A


def create_C_array(bdata, n_buses):
    n = bdata.shape[0]
    C = np.zeros((n_buses, n))
    for i in range(n_buses):
        for j in range(n):
            if bdata[j, 1] == i+1:
                C[i, j] = 1
    return C

def create_B_array(bdata):
    n = bdata.shape[0]
    B = np.zeros((n, n))
    for i in range(n):
        B[i, i] = 1
    return B

def power_from_vi(vars, A, B, C):
    Pff_v1  = tf.linalg.matmul(vars[0], A)
    Pff_v2  = tf.linalg.matmul(vars[2], B)
    Pff_v3  = tf.linalg.matmul(vars[1], A)
    Pff_v4  = tf.linalg.matmul(vars[3], B)

    Pft_v1  = tf.linalg.matmul(vars[0], C)
    Pft_v2  = tf.linalg.matmul(vars[4], B)
    Pft_v3  = tf.linalg.matmul(vars[1], C)
    Pft_v4  = tf.linalg.matmul(vars[5], B)


    PFF = tf.multiply(tf.add(tf.multiply(Pff_v1, Pff_v2), tf.multiply(Pff_v3, Pff_v4)), 100)
    PFT = tf.multiply(tf.add(tf.multiply(Pft_v1, Pft_v2), tf.multiply(Pft_v3, Pft_v4)), 100)

    return PFF, PFT




def get_errors(X_v, Y_p, dnn, mse, mape, r2):
    _, x_test, _, y_test = train_test_split(X_v, Y_p, test_size=0.2)

    x_test_scaled = dnn.x_scaler.transform(x_test)

    yhat = tf.convert_to_tensor(dnn.model.predict(x_test), dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


    bdata = np.genfromtxt(r"System_data\bdata.csv", delimiter=',')
    A = tf.convert_to_tensor(create_A_array(bdata,118), dtype=tf.float32)
    B = tf.convert_to_tensor(create_B_array(bdata), dtype=tf.float32)
    C = tf.convert_to_tensor(create_C_array(bdata, 118), dtype=tf.float32)


    lengths = [118, 118, 186,186,186,186] # VR, VI, CFR, CFI, CTR, CTI

    sample_input = yhat
    vars = tf.split(sample_input, lengths, axis=1)


    vars_actual = tf.split(y_test, lengths, axis=1)

    PFF_actual ,PFT_actual = power_from_vi(vars_actual, A, B, C)
    PFF, PFT = power_from_vi(vars, A, B, C)

    # Compute squared difference
    squared_difference_PFF = tf.square(tf.subtract(PFF, PFF_actual))
    squared_difference_PFT = tf.square(tf.subtract(PFT, PFT_actual))

    # Compute mean
    mean_squared_error_PFF = tf.reduce_mean(squared_difference_PFF)
    mean_squared_error_PFT = tf.reduce_mean(squared_difference_PFT)

    # Compute square root to get RMSE
    rmse_PFF = tf.sqrt(mean_squared_error_PFF)
    rmse_PFT = tf.sqrt(mean_squared_error_PFT)

    # print("Root Mean Squared Error for PFF: ", rmse_PFF)
    # print("Root Mean Squared Error for PFT: ", rmse_PFT)
    
    bflow_hat = reorder(PFF, PFT)
    bflow_actual = reorder(PFF_actual, PFT_actual)
    y = tf.linalg.matmul(bflow_actual, dnn.last_layer_weights2)
    yhat = tf.linalg.matmul(bflow_hat, dnn.last_layer_weights2)
    pinj_error = tf.sqrt(tf.reduce_mean((y-yhat)**2))
    pflowerror = (rmse_PFF+rmse_PFT)/2
    v_error1 = tf.sqrt(tf.reduce_mean((vars[0]-vars_actual[0])**2))
    v_error2 = tf.sqrt(tf.reduce_mean((vars[1]-vars_actual[1])**2))
    v_error = (v_error1+v_error2)/2
    r2_filtered = [x if -1<=x<=1 else 0 for x in r2[0]]
    np.mean(r2_filtered)
    return v_error, pflowerror, pinj_error, r2_filtered



Root Mean Squared Error for PFF:  tf.Tensor(10.383515, shape=(), dtype=float32)
Root Mean Squared Error for PFT:  tf.Tensor(10.375895, shape=(), dtype=float32)


Calculate power injection error

tf.Tensor(11.899157, shape=(), dtype=float32)


Make loooooop

In [13]:
pmu_loc = {
    11: [8, 9, 10, 26, 30, 38, 63, 64, 65, 68, 81] ,
    13: [8, 9, 10, 26, 30, 38, 63, 64, 65, 68, 81, 87, 111] ,
    32: [1, 5, 9, 12,13,17,21,23,26,28,34,37,41,45,49,53,56,62,63,68,71,75,77,80,85,86,90,94,101,105,110,114],
    118: np.arange(1,119)
}

add_bus = [x for x in pmu_loc[32] if x not in pmu_loc[11]]

v_error = []
i_error = []
pflow_error = []
pinj_error = []

for additional_bus in add_bus:
    

28


In [9]:
# import tensorflow as tf

# def create_A_array(bdata, n_buses):
#     n = bdata.shape[0]
#     A = np.zeros((n_buses, n))
#     for i in range(n_buses):
#         for j in range(n):
#             if bdata[j, 0] == i+1:
#                 A[i, j] = 1
#     return A


# def create_C_array(bdata, n_buses):
#     n = bdata.shape[0]
#     C = np.zeros((n_buses, n))
#     for i in range(n_buses):
#         for j in range(n):
#             if bdata[j, 1] == i+1:
#                 C[i, j] = 1
#     return C

# def create_B_array(bdata):
#     n = bdata.shape[0]
#     B = np.zeros((n, n))
#     for i in range(n):
#         B[i, i] = 1
#     return B


# A = tf.convert_to_tensor(create_A_array(bdata,118), dtype=tf.float32)
# B = tf.convert_to_tensor(create_B_array(bdata), dtype=tf.float32)
# C = tf.convert_to_tensor(create_C_array(bdata, 118), dtype=tf.float32)

# X,y = data([], 'vi_all')


In [10]:
# # sample_input = y[0,:]
# # sample_output = tf.convert_to_tensor(bflow.values[0,:]) # optimize this
# # sample_input.shape

# sample_input = tf.random.normal([64, 980], dtype=tf.float32)

# lengths = [118, 118, 186,186,186,186] # VR, VI, CFR, CFI, CTR, CTI

# vars = tf.split(sample_input, lengths, axis=1)

# Pff_v1  = tf.linalg.matmul(vars[0], A)
# Pff_v2  = tf.linalg.matmul(vars[2], B)
# Pff_v3  = tf.linalg.matmul(vars[1], A)
# Pff_v4  = tf.linalg.matmul(vars[3], B)

# Pft_v1  = tf.linalg.matmul(vars[0], C)
# Pft_v2  = tf.linalg.matmul(vars[4], B)
# Pft_v3  = tf.linalg.matmul(vars[1], C)
# Pft_v4  = tf.linalg.matmul(vars[5], B)


# PFF = tf.multiply(tf.add(tf.multiply(Pff_v1, Pff_v2), tf.multiply(Pff_v3, Pff_v4)), 100)
# PFT = tf.multiply(tf.add(tf.multiply(Pft_v1, Pft_v2), tf.multiply(Pft_v3, Pft_v4)), 100)


# ePF = PFF
# ePT = PFT # change this

# pfl_vr_1 = np.zeros([sample_input.shape[0], lengths[0]])
# pfl_vr_2 = np.zeros([sample_input.shape[0], lengths[0]])
# pfl_vi_1 = np.zeros([sample_input.shape[0], lengths[1]])
# pfl_vi_2 = np.zeros([sample_input.shape[0], lengths[1]])
# pfl_cfr = np.zeros([sample_input.shape[0], lengths[2]])
# pfl_cfi = np.zeros([sample_input.shape[0], lengths[3]])
# pfl_ctr = np.zeros([sample_input.shape[0], lengths[4]])
# pfl_cti = np.zeros([sample_input.shape[0], lengths[5]])


# for i in range(bdata.shape[0]):
#     # current index is i, so is power 
#     fbus = int(bdata[i,0])-1
#     tbus = int(bdata[i,1])-1

#     # loss of fbus updated with i-th from current
#     pfl_vr_1[:,fbus] = pfl_vr_1[:,fbus] + ePF[:,i]*vars[2][:,i]/(1+ vars[2][:,i]**2)
#     pfl_vi_1[:,fbus] = pfl_vi_1[:,fbus] + ePF[:,i]*vars[3][:,i]/(1+ vars[3][:,i]**2)
    
#     # loss of tbus updated with ith to current
#     pfl_vr_2[:,tbus] = pfl_vr_2[:,tbus] + ePT[:,i]*vars[4][:,i]/(1+vars[4][:,i]**2)
#     pfl_vi_2[:,tbus] = pfl_vi_2[:,tbus] + ePT[:,i]*vars[5][:,i]/(1+vars[5][:,i]**2)

#     # loss of ith current updated with fbus voltage and tbus voltage
#     pfl_cfr[:,i] = pfl_cfr[:,i] + ePF[:,i]*vars[0][:,fbus]/(1+vars[0][:,fbus]**2)

#     pfl_cfi[:,i] = pfl_cfi[:,i] + ePF[:,i]*vars[1][:,fbus]/(1+vars[1][:,fbus]**2)

#     pfl_ctr[:,i] = pfl_ctr[:,i] + ePF[:,i]*vars[0][:,tbus]/(1+vars[0][:,tbus]**2)

#     pfl_cti[:,i] = pfl_cti[:,i] + ePF[:,i]*vars[1][:,tbus]/(1+vars[1][:,tbus]**2)


# vr_loss = tf.convert_to_tensor(pfl_vr_1 + pfl_vr_2)
# vi_loss = tf.convert_to_tensor(pfl_vi_1 + pfl_vi_2)

# cfr_loss = tf.convert_to_tensor(pfl_cfr)
# cfi_loss = tf.convert_to_tensor(pfl_cfi)
# ctr_loss = tf.convert_to_tensor(pfl_ctr)
# cti_loss = tf.convert_to_tensor(pfl_cti)

# loss_for_sample_b = tf.concat([vr_loss, vi_loss, cfr_loss, cfi_loss, ctr_loss, cti_loss], axis=1)
# #     total_loss.append(loss_for_sample_b)

# # # Convert list of tensors to tensor
# # total_loss = tf.concat(total_loss, axis=0)